In [14]:
import numpy as np
import pandas as pd

### Load ATLAS Data

#### Acceptance

In [15]:
tauDict = {'stau': {0.2 : (9,70), 1.0 : (26,56), 3.0 : (43,42), 10.0 : (60,28), 30.0 : (77,14), 1e10 : (94,0)}
         }
           

dfs = []
files = {'stau' : './HEPData-ins2080541-v1-csv/Acceptance,Stau.csv'}
for tag in tauDict:
    for tau,(skip_header,skip_footer) in tauDict[tag].items():
        data = np.genfromtxt(files[tag],skip_header=skip_header,delimiter=',',names=True,skip_footer=skip_footer)
        newDF = pd.DataFrame.from_dict({'mLLP' : data['Mass_GeV']})
        newDF['Acceptance'] = data['Acceptance']
        newDF['tau_ns'] = tau
        dfs.append(newDF)

atlasAccDF = pd.concat(dfs, ignore_index=True)

#### Event Level Efficiency

In [16]:
dfs = []
files = {'stau' : './HEPData-ins2080541-v1-csv/Event-levelefficiency,Stau.csv'}
for tag in tauDict:
    for tau,(skip_header,skip_footer) in tauDict[tag].items():
        data = np.genfromtxt(files[tag],skip_header=skip_header,delimiter=',',names=True,skip_footer=skip_footer)
        newDF = pd.DataFrame.from_dict({'mLLP' : data['Mass_GeV']})
        newDF['Event Efficiency'] = data['Eventlevel_Efficiency']
        newDF['tau_ns'] = tau
        dfs.append(newDF)

atlasEvtEffDF = pd.concat(dfs, ignore_index=True)

#### SR Efficiency

In [17]:
atlasSRs = []
for sr in ['Inclusive_Low','Inclusive_High']:
    sr_tag = sr.split('_')[1]
    dfs = []
    files = {'stau' : './HEPData-ins2080541-v1-csv/Efficiency,SR-%s,Stau.csv'%sr}
    for tag in tauDict:
        for tau,(skip_header,skip_footer) in tauDict[tag].items():
            data = np.genfromtxt(files[tag],skip_header=skip_header,delimiter=',',names=True,skip_footer=skip_footer)
            newDF = pd.DataFrame.from_dict({'mLLP' : data['Mass_GeV']})
            newDF['(SR-%s - no mass Window)' %sr_tag] = data['Efficiency_%s' %sr]
            newDF['tau_ns'] = tau
            dfs.append(newDF)

    atlasSRs.append(pd.concat(dfs, ignore_index=True))
atlasSREffDF = pd.merge(*atlasSRs)

In [18]:
atlasDF = pd.merge(atlasAccDF, atlasEvtEffDF,how='inner')
atlasDF = pd.merge(atlasDF, atlasSREffDF,how='inner')

In [19]:
orderColumns = ['mLLP','tau_ns']
atlasDF.sort_values(orderColumns,inplace=True,
                ascending=[True,True],ignore_index=True)
res = atlasDF.reindex(columns=orderColumns+[c for c in atlasDF.columns if c not in orderColumns])

In [20]:
atlasDF.to_pickle('stauTables.pcl')